In [1]:
import pandas as pd
import json

with open('뜻을 공유하고 소통.json', 'r') as file:
    data = json.load(file)

In [13]:
# 기본 형식 점수 계산

def is_real_VP(morp_result):
    return (
        'VX' not in morp_result or
        'VV' in morp_result or
        'VA' in morp_result or
        'VCP' in morp_result or
        'VCN' in morp_result
    )

basic_per_sent= []
for sentence in data['return_object']['sentence']:
    basic_per_clause = []
    dep_id2label = {int(dep['id']): dep['label'] for dep in sentence['dependency']}
    for morp, dep in zip(sentence['morp_eval'], sentence['dependency']):
        if ('VP' in dep['label']) or ('VNP' in dep['label']):
            if is_real_VP(morp['result']):
                basic_elements = [dep_id2label[int(sub)] for sub in dep['mod'] if int(sub) in dep_id2label]
                basic_per_clause.append(basic_elements)
    basic_per_sent.append(basic_per_clause)

basic_score_per_sent = []
for sent in basic_per_sent:
    basic_score = 0
    for clause in sent:
        basic_score_per_clause = 1
        if ('VP_OBJ' in clause) or ('NP_OBJ' in clause) or ('X_OBJ' in clause) or ('VNP_OBJ' in clause):
            basic_score_per_clause += 1
        if ('AP_CMP' in clause) or ('X_CMP' in clause) or ('NP_CMP' in clause) or ('VP_CMP' in clause) or ('VNP_CMP' in clause):
            basic_score_per_clause += 1
        basic_score += basic_score_per_clause
    # 최대 점수 제한 기능(삭제를 원할 경우 아래 4줄 삭제)
    if basic_score > 20:
        basic_score = 20
    else:
        basic_score = basic_score
    basic_score_per_sent.append(basic_score)

basic_score_per_sent

[8, 7]

In [ ]:
# 수식 구조 점수 계산

add1_score_per_sent = []

for sentence in data['return_object']['sentence']:
    add1_score = 0
    for dep in sentence['dependency']:
        if dep['label'] in ['NP_MOD', 'X_MOD', 'DP', 'NP_AJT', 'AP_AJT', 'X_AJT', 'NP_INT', 'IP']:
            add1_score += 1
        if dep['label'] =='AP':
            if dep['text'] not in ['없이', '달리', '같이']:
                add1_score += 1
            elif len(dep['mod']) == 0:
                add1_score += 1
        if dep['label'] in ['NP_SBJ', 'NP_OBJ', 'NP_CMP', 'NP_MOD', 'NP_CNJ', 'NP', 'NP_AJT', 'NP_INT']:
            add1_score += dep['mod'].count('NP')
    # 최대 점수 제한 기능(삭제를 원할 경우 아래 4줄 삭제)
    if add1_score >= 14:
        add1_score = 14
    else:
        add1_score = add1_score
    add1_score_per_sent.append(add1_score/2)

add1_score_per_sent

[0.5, 0.5]

In [ ]:
# 내포 구조 점수 계산

clause_list = ['VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']

add2_score_per_sent = []
for sentence in data['return_object']['sentence']:
    add2_score = 0
    for morp, dep in zip(sentence['morp_eval'], sentence['dependency']):
        # 명사절
        if (dep['label'] in ['VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ']):
            # 한 단어
            if len(dep['mod']) == 0:
                add2_score += 1
            else:
                add2_score += 2
                # 내포절 안의 내포절
                pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                if x['mod'].count('VP_SBJ') + x['mod'].count('VNP_SBJ') + x['mod'].count('X_SBJ') + x['mod'].count('NP_SBJ') >= 2:
                                    add2_score += 1

        # 인용절
        if (dep['label'] in ['VP_CMP', 'VNP_CMP']):
            # 한 단어
            if len(dep['mod']) == 0:
                add2_score += 1
            else:
                add2_score += 2
            # 내포절 안의 내포절
                pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                if x['mod'].count('VP_SBJ') + x['mod'].count('VNP_SBJ') + x['mod'].count('X_SBJ') + x['mod'].count('NP_SBJ') >= 2:
                                    add2_score += 1

        # 관형절
        if (dep['label'] in ['VP_MOD', 'VNP_MOD']):
            # 한 단어
            if len(dep['mod']) == 0:
                add2_score += 1
            else:
                add2_score += 2
                # 내포절 안의 내포절
                pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                if x['mod'].count('VP_SBJ') + x['mod'].count('VNP_SBJ') + x['mod'].count('X_SBJ') + x['mod'].count('NP_SBJ') >= 2:
                                    add2_score += 1

        # 부사절
        if (dep['label'] in ['VP_AJT', 'VNP_AJT']):
            # 한 단어
            if len(dep['mod']) == 0:
                add2_score += 1
            else:
                add2_score += 2
                # 내포절 안의 내포절
                pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                if x['mod'].count('VP_SBJ') + x['mod'].count('VNP_SBJ') + x['mod'].count('X_SBJ') + x['mod'].count('NP_SBJ') >= 2:
                                    add2_score += 1

            # 부사절(달리, 없이, 같이)
        if (dep['label'] in ['AP']) & (dep['text'] in ['달리', '없이', '같이']) & (len(dep['mod']) >= 1):
            add2_score += 2
            # 내포절 안의 내포절
            pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
            for clause in clause_list:
                add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
            for num, elem in zip(dep['mod'], pre_add2_elements):
                if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                    for x in sentence['dependency']:
                        if int(num) == int(x['id']):
                            if x['mod'].count('VP_SBJ') + x['mod'].count('VNP_SBJ') + x['mod'].count('X_SBJ') + x['mod'].count('NP_SBJ') >= 2:
                                add2_score += 1

            # 서술절
        if (dep['label'] in ['VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']):
            pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
            if pre_add2_elements.count('VP_SBJ') + pre_add2_elements.count('VNP_SBJ') + pre_add2_elements.count('X_SBJ') + pre_add2_elements.count('NP_SBJ') >= 2:
                add2_score += 2
            # 내포절 안의 내포절
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                if x['mod'].count('VP_SBJ') + x['mod'].count('VNP_SBJ') + x['mod'].count('X_SBJ') + x['mod'].count('NP_SBJ') >= 2:
                                    add2_score += 1
    # 최대 점수 제한 기능(삭제를 원할 경우 아래 4줄 삭제)
    if add2_score > 17:
        add2_score = 17
    else:
        add2_score = add2_score
    add2_score_per_sent.append(add2_score)

add2_score_per_sent


[4, 10]

In [ ]:
# 접속 구조 점수 계산

add3_score_per_sent = []

for sentence in data['return_object']['sentence']:
    add3_score = 0
    for morp, dep, num in zip(sentence['morp_eval'], sentence['dependency'], range(len(sentence['dependency']))):
        if (dep['label'] in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN']) and (dep['head'] != -1):
            if ('VX' not in morp['result']) or ('VV' in morp['result']) or ('VA' in morp['result']) or ('VCP' in morp['result']) or ('VCN' in morp['result']):
                if num < len(sentence['morp_eval'])-1:
                    if ('VX' in sentence['morp_eval'][num+1]['result']) and ('VV' not in sentence['morp_eval'][num+1]['result']) and ('VA' not in sentence['morp_eval'][num+1]['result']) and ('VCP' not in sentence['morp_eval'][num+1]['result']) and ('VCN' not in sentence['morp_eval'][num+1]['result']):
                        if (sentence['dependency'][num+1]['label'] in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN']) and (sentence['dependency'][num+1]['head'] != -1):
                            add3_score += 1
                    else:
                        add3_score += 1
    # 최대 점수 제한 기능(삭제를 원할 경우 아래 4줄 삭제)
    if add3_score > 7:
        add3_score = 7
    else:
        add3_score = add3_score
    add3_score_per_sent.append(add3_score)

add3_score_per_sent


[3, 0]

In [16]:
# 합산 점수 계산
sum_score_per_sent = [a + b + c + d for a, b, c, d in zip(basic_score_per_sent, add1_score_per_sent, add2_score_per_sent, add3_score_per_sent)]
sum_score_per_sent

[15.5, 17.5]

In [20]:
sentence_list = [x['text'] for x in data['return_object']['sentence']]
df_result_final = pd.DataFrame({'문장': sentence_list,
                                '기본 형식': basic_score_per_sent, 
                                '수식': add1_score_per_sent,
                                '내포': add2_score_per_sent,
                                '접속': add3_score_per_sent, 
                                '합산': sum_score_per_sent})

df_result_final

,문장,기본 형식,수식,내포,접속,합산
0,"뜻을 공유하고 소통하는 관계인지, 아니면 다른 무엇인지 궁금해진다.",8,0.5,4,3,15.5
1,하지만 사람이 하는 일이라 알 수 있겠나”라고 말했다.,7,0.5,10,0,17.5


In [21]:
df_result_final.to_excel('df_result.xlsx')